# **Installation of Packages**

First install packages like numpy, scikit-learn, matplotlib

In [1]:
!pip install numpy scikit-learn matplotlib

# **Importation of packages**

We import the necessary packages

In [2]:
import os, sys, json
import urllib 
import shutil
import numpy
from SPARQLWrapper import SPARQLWrapper, JSON
from PIL import Image
from sklearn.cluster import KMeans

# **Load Dataset**

In [7]:
endpoint_url = "https://query.wikidata.org/sparql"

img_data = {}

#Requete pour recuperer les infos de l'entité donnée - Q144/Chien par defaut
def query(wd='Q144', limit=5):
     return """SELECT ?item ?itemLabel ?pic{?item wdt:P31 wd:"""+wd+""".?item wdt:P18 ?pic}limit """+str(limit)+""""""

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

#Pour creer le fichier des images s'il existe pas
def createFolder(name='images'):
    if name not in os.listdir():
        os.mkdir(name)
    else:
        print('Dossier '+name+' déjà exitant !')

#Pour verifier si un dossier est vide - evitons de retélécharger des images inutilement.
def isEmpty(name='images'):
    if len(os.listdir(name)) == 0:
        return True
    return False

#Pour ajouter des elements a la bdd
def addImg(wd,limit):
    results = get_results(endpoint_url, query(wd,limit))
    for result in results['results']['bindings']:
        img_data[result['item']['value'].split("/")[-1]] = {'image': result['pic']['value'],}
    #print(img_data)

def addAllImg(wd,limit):
    for w in wd:
        addImg(w,limit)

#Pour telecharger l'image d'un element de la bd
def downloadImage(url, name_img='img0'):
    full_path = 'images/'+name_img+'.jpg'
    urllib.request.urlretrieve(url, full_path)

#Si les images sont deja telechargées on initialise le tableau
def initAlreadyDownload(namefolder='images'):
    for image in os.listdir(namefolder):
        img_data[image.split(".")[0]] = {'image': "",}
    
#Pour télécharger toutes les images - limit par query
def downloadAllImages(wd=['Q144'], limit=5):
    createFolder()
    if isEmpty() == False:
        print('Images déjà téléchargées !')
        initAlreadyDownload()
        return False
    addAllImg(wd,limit)
    for i,img in enumerate(img_data):
        downloadImage(img_data[img]['image'], list(img_data.keys())[i])
    return True 


#Supprimer une image du dossier 
def removeImage(image):
    if os.path.isfile('images/'+image):
        os.remove('images/'+image)
        img_data.pop(image.split(".")[0])
    else:
        shutil.rmtree('images/'+image)
    

        
downloadAllImages(wd=['Q144','Q14660'],limit=5)
print(img_data)


Dossier images déjà exitant !
Images déjà téléchargées !
{'Q4485657': {'image': ''}, 'Q5515388': {'image': ''}, 'Q5859102': {'image': ''}, 'Q5375926': {'image': ''}, 'Q5619202': {'image': ''}, 'Q5686986': {'image': ''}, 'Q4485634': {'image': ''}, 'Q4485622': {'image': ''}, 'Q4485652': {'image': ''}, 'Q4485643': {'image': ''}}


# **Adding Meta to the Dataset**

## *1. Exif* 

In [8]:
#Obtenir les metas d'une image
def getMetaImg(name, imgfile): #name = 'img'
    img_data[name]['taille'] = imgfile.size
    img_data[name]['format'] = imgfile.format
    getColorsImg(name, imgfile)
    
    exif_data = imgfile._getexif()
    if exif_data:
        img_data[name]['orientation'] = exif_data.get(274, None) #Orientation de l'image
        """
        1: l'image est en orientation normale (pas de rotation nécessaire).
        2: l'image est miroir horizontale (il faut la tourner de 180 degrés).
        3: l'image est en orientation 180 degrés (il faut la tourner de 180 degrés).
        4: l'image est miroir verticale (il faut la tourner de 270 degrés).
        5: l'image est en orientation 90 degrés dans le sens horaire et miroir horizontale (il faut la tourner de 90 degrés dans le sens horaire).
        6: l'image est en orientation 90 degrés dans le sens horaire (il faut la tourner de 90 degrés dans le sens horaire).
        7: l'image est en orientation 90 degrés dans le sens anti-horaire et miroir horizontale (il faut la tourner de 90 degrés dans le sens anti-horaire).
        8: l'image est en orientation 90 degrés dans le sens anti-horaire (il faut la tourner de 90 degrés dans le sens anti-horaire).
        """
        img_data[name]['date'] = exif_data.get(36867, "") #Date
        img_data[name]['model'] = exif_data.get(272, "") #Appareil utilisé
    else:
        print("Aucune donnée Exif trouvée.")



## *2. Main Color* 

In [9]:
def getColorsImg(name, imgfile, cluster_number=4, n_init=5):
    colors = []
    numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    numarray = numarray.reshape(-1, 3)
    clusters = KMeans(n_clusters=cluster_number, n_init=n_init)
    clusters.fit(numarray)
    for i in range(cluster_number):
        colors.append((clusters.cluster_centers_[i][0],
                        clusters.cluster_centers_[i][1],
                        clusters.cluster_centers_[i][2]))
    img_data[name]['colors'] = {i:colors[i] for i in range(len(colors))}


## *3. Export to JSON* 

In [10]:
def openImgGetMeta(name): #name = 'img.jpg'
    try:
        with Image.open("images/"+name) as imgfile:
            #print(imgfile.size, imgfile.format)
            name = name.split(".")[0]
            getMetaImg(name, imgfile)

        imgfile.close()
        return True
    except FileNotFoundError as e:
        print('Impossible de trouver le fichier {}: {}'.format("images/"+name, e))
        return False
    except Exception as e:
        print('Erreur lors de l\'ouverture de l\'image {}: {}'.format("images/"+name, e))
        removeImage(name)
        return False
        
#Obtenir les metas de toutes nos images    
def openGetMetaAllImg(namefolder='images'):
    for image in os.listdir(namefolder):
        openImgGetMeta(image)

openGetMetaAllImg()



#Enregistre les metadonnées des images dans un JSON
def exportToJSON(name='img_data.json'):
    with open(name, 'w', encoding='utf-8') as f:
        json.dump(img_data, f, ensure_ascii=False, indent=4)
        
exportToJSON()
  


Aucune donnée Exif trouvée.
Aucune donnée Exif trouvée.
Aucune donnée Exif trouvée.
Aucune donnée Exif trouvée.
Aucune donnée Exif trouvée.
Aucune donnée Exif trouvée.
Aucune donnée Exif trouvée.
